In [1]:
import os
os.chdir("../")

In [2]:
from storage.har_datasets import STSDataset, StreamingTimeSeries, StreamingTimeSeriesCopy, HARTHDataset
from s3ts.dtw_layer.lightningstsds import LSTSDataset
from s3ts.dtw_layer.dtw_layer import DTWLayer
from s3ts.dtw_layer.wrapper import lmodel
from pytorch_lightning import Trainer
import numpy as np
import torch

In [3]:
label_mapping = np.zeros(141)
label_mapping[1:9] = np.arange(8)
label_mapping[13] = 8
label_mapping[14] = 9
label_mapping[130] = 10
label_mapping[140] = 11

ds = HARTHDataset("./datasets/HARTH/", wsize=80, normalize=True, label_mapping=label_mapping)
print(len(ds))

6205301


In [4]:
data_split = {
    "train": lambda x: x<6000000,
    "val": lambda x: (x>=6000000) * (x<6050000),
    "test": lambda x: x>=6050000
}

dm = LSTSDataset(ds, data_split=data_split, batch_size=128, random_seed=42, num_workers=8)

In [5]:
model = torch.nn.Sequential(
    DTWLayer(8, 6, 48, l_out=48, rho=0.01),
    torch.nn.Conv2d(in_channels=8, out_channels=32, kernel_size=5), # out 44x44
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=2), # out 22x22
    torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5), # out 18x18
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=2), # out 9x9
    torch.nn.Conv2d(in_channels=64, out_channels=128, kernel_size=4), # out 6x6 
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(kernel_size=2), # out 3x3
    torch.nn.Flatten(),
    torch.nn.Linear(in_features=128*9, out_features=12)
)

In [6]:
trainer = Trainer(max_epochs=50, accelerator="auto")
model_lightning = lmodel(model, 12, lr=0.001)
trainer.fit(model=model_lightning, datamodule=dm)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name       | Type               | Params
--------------------------------------------------
0 | model      | Sequential         | 205 K 
1 | softmax    | Softmax            | 0     
2 | train_acc  | MulticlassAccuracy | 0     
3 | train_f1   | MulticlassF1Score  | 0     
4 | val_acc    | MulticlassAccuracy | 0     
5 | val_f1     | MulticlassF1Score  | 0     
6 | val_auroc  | MulticlassAUROC    | 0     
7 | test_acc   | MulticlassAccuracy | 0     
8 | test_f1    | MulticlassF1Score  | 0     
9 | test_auroc | MulticlassAUROC    | 0     
--------------------------------------------------
205 K     Trainable params
0         Non-trainable params
205 K     Total params
0.820     Total estimated model params size (MB)


Epoch 0:   0%|          | 90/46486 [00:36<5:12:04,  2.48it/s, v_num=40, train_loss_step=0.698]

In [6]:
from time import time

In [7]:
@torch.jit.script
def dtw_compute_faster_time(dist_tensor: torch.Tensor, grad_tensor: torch.Tensor, w: float) -> None:
    for i in range(1, dist_tensor.shape[0]):
        for j in range(1, dist_tensor.shape[1]):
            # elements has shape (3, n, k)
            value = torch.minimum(w * torch.minimum(dist_tensor[i, j-1], dist_tensor[i-1, j-1]), dist_tensor[i-1, j])
            id = (w * dist_tensor[i, j-1] < dist_tensor[i-1, j]) & (dist_tensor[i, j-1] < dist_tensor[i-1, j-1])

            # value, id = torch.minimum((w * dist_tensor[i, j-1], dist_tensor[i-1, j], w * dist_tensor[i-1, j-1]), dim=0).min(dim=0) # shape (n, k)

            dist_tensor[i, j] += value

            grad_tensor[id][:, :, i, j] += w * grad_tensor[id][:, :, i, j-1]

def torch_dtw_faster_time(x: torch.Tensor, y: torch.Tensor, w: float, eps: float = 1e-5):
    a=time()
    # shape of x (n, dim, x_len) y (m, dim, y_len)

    # performs convolution-like operation, for each kernel the DF
    # (of shape (kernel_size, T)) is computed, then summed across channels
    # x has shape (batch, c, time_dimension)

    # compute pairwise diffs (squared)
    p_diff = x[:,None,:,None,:] - y[None,:,:,:,None] # shape (n, n_kernel, d, Kernel_size, T)
    euc_d = torch.square(p_diff).sum(2) # shape (n, n_kernel, kernel_size, T)

    # p_diff contains the partial derivatives of DTW[n, k, i, j] wrt K[k, d, i] (dims (n, k, d, i, j))
    p_diff = p_diff / torch.sqrt(euc_d[:,:, None, :, :] + eps)

    # compute dtw
    euc_d[:,:,0,:] = torch.cumsum(euc_d[:,:,0,:], dim=2)
    euc_d[:,:,:,0] = torch.cumsum(euc_d[:,:,:,0], dim=2)

    # rearrange dims
    DTW = torch.permute(euc_d, (2, 3, 0, 1)).contiguous()
    b=time()
    dtw_compute_faster_time(DTW, p_diff, w)
    print(time()-a)
    # recover dimensions
    DTW = torch.permute(DTW, (2, 3, 0, 1)).contiguous()
    print(time()-a)
    return DTW.sqrt(), p_diff

In [8]:
x = torch.randn(128, 6, 128)
y = torch.randn(16, 6, 64)

In [9]:
from s3ts.dtw_layer.dtw import torch_dtw_fast


In [15]:
torch_dtw_faster_time(x, y, 0.98)

0.467998743057251
0.5760643482208252


(tensor([[[[ 1.7794,  2.4011,  4.6142,  ..., 40.1241, 40.2339, 40.3594],
           [ 3.8315,  3.7152,  4.2538,  ..., 23.3262, 23.1583, 23.0341],
           [ 5.2223,  4.7721,  5.2265,  ..., 23.4899, 23.3300, 23.1588],
           ...,
           [27.1420, 25.0323, 22.9254,  ..., 18.4274, 18.3333, 18.2747],
           [27.2536, 25.1779, 23.2396,  ..., 18.8065, 18.5357, 18.6725],
           [27.6731, 25.5884, 23.3907,  ..., 18.8907, 18.9362, 18.6684]],
 
          [[ 4.1024,  5.4241,  5.9583,  ..., 35.3831, 35.5243, 35.6050],
           [ 4.7649,  4.7884,  5.5179,  ..., 22.8626, 22.7701, 22.8075],
           [ 6.3059,  6.3672,  5.3783,  ..., 22.7898, 22.8942, 22.7825],
           ...,
           [29.4742, 27.1925, 25.7042,  ..., 19.0534, 18.9629, 18.8095],
           [29.5225, 27.2218, 25.9137,  ..., 19.2199, 19.0964, 19.1080],
           [29.7935, 27.5202, 25.9628,  ..., 18.8077, 18.8878, 18.8977]],
 
          [[ 3.3421,  4.3751,  5.4556,  ..., 35.7847, 35.8954, 35.9594],
           [ 